<a href="https://colab.research.google.com/github/marreapato/Deep_Learning_Course/blob/main/all_labels_VLModel_Lucas_Rabelo_modelo_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificador zero-shot utilizando o prompt da categoria.


In [ ]:
!pip install transformers

## Modelo CLIP

In [ ]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:

model.text_model.embeddings

CLIPTextEmbeddings(
  (token_embedding): Embedding(49408, 512)
  (position_embedding): Embedding(77, 512)
)

#Connect with Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Fundamentals_Deep_Learning

!ls




/content/gdrive/.shortcut-targets-by-id/1RQft65Bgd2d6TCEfL0c3FLA4gzbYIqmV/Fundamentals_Deep_Learning
 001_tensorflow.ipynb		     BANKEX.csv
 00_jupyterlab.ipynb		    'Cópia de 01_mnist.ipynb'
 01_mnist.ipynb			     data
 02_asl.ipynb			     GRU_B20
 03_asl_cnn.ipynb		     images
 04a_asl_augmentation.ipynb	     LSTM_B20
 04b_asl_predictions.ipynb	     mit_indoor_scene_dataset
 05a_doggy_door.ipynb		    'NN FeedForward - CIFAR - 10.ipynb'
 05b_presidential_doggy_door.ipynb   original
 06_headline_generator.ipynb	     outdir
 activities.csv			     RNN_multi.ipynb
 asl_model


In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# For separating train and test sets
from sklearn.model_selection import train_test_split

# For visualizations
import matplotlib.pyplot as plt
import matplotlib.image as img
import PIL.Image as Image
import cv2

import os
import numpy as np
import pathlib

data_dir = "/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/"
data_dir = pathlib.Path(data_dir)


train_dataset = list(data_dir.glob('train/*'))
test_dataset = list(data_dir.glob('test/*'))

In [ ]:
train_dataset

[PosixPath('/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/train/ship'),
 PosixPath('/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/train/equine'),
 PosixPath('/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/train/deer'),
 PosixPath('/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/train/bird'),
 PosixPath('/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/train/plane'),
 PosixPath('/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/train/cat'),
 PosixPath('/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/train/car'),
 PosixPath('/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/train/dog'),
 PosixPath('/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/train/truck'),
 PosixPath('/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/train/frog')]

In [ ]:
from pathlib import Path
# List to store (label, image, caption) tuples
labeled_images = []

# Loop through each folder and open each image
for folder_path in train_dataset:
    folder_path = Path(folder_path)

    # Extract label from folder name
    label = folder_path.stem

    # List all files in the folder
    image_files = folder_path.glob('*.*')

    # Loop through each image file
    for image_file in image_files:
        # Open the image
        image = Image.open(image_file)

        # Create a standard caption with the label at the end
        caption = f"Standard Caption Here - Label: {label}"

        # Append the (label, image, caption) tuple to the list
        labeled_images.append((label, image, caption))

In [ ]:
# Extract images from the labeled_images tuple using a list comprehension
images_vector = [item[1] for item in labeled_images]

In [ ]:
images_vector

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.Jpeg

In [ ]:
# Extract images from the labeled_images tuple using a list comprehension
captions_vector = [item[2] for item in labeled_images]

In [ ]:
images_vector[0:3]

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>]

In [ ]:
inputs = processor(
    text=["this is a ship","this is a box","this is a cat"],
    images=images_vector[0:3],
    return_tensors="pt",
    padding=True
    )

In [ ]:
inputs

{'input_ids': tensor([[49406,   589,   533,   320,  1158, 49407],
        [49406,   589,   533,   320,  2063, 49407],
        [49406,   589,   533,   320,  2368, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]]), 'pixel_values': tensor([[[[ 0.2223,  0.2223,  0.2369,  ...,  0.1785,  0.1639,  0.1639],
          [ 0.2223,  0.2223,  0.2369,  ...,  0.1639,  0.1493,  0.1493],
          [ 0.2077,  0.2077,  0.2223,  ...,  0.1201,  0.1201,  0.1055],
          ...,
          [-0.6974, -0.5368, -0.7558,  ..., -0.6244, -0.8288, -0.5076],
          [ 0.3683,  0.4705,  0.0763,  ..., -0.7266, -0.9456, -0.5368],
          [ 0.1785,  0.2077,  0.2807,  ..., -0.6974, -0.9310, -0.4930]],

         [[ 0.4991,  0.4991,  0.5141,  ...,  0.4540,  0.4390,  0.4390],
          [ 0.4991,  0.4991,  0.5141,  ...,  0.4390,  0.4090,  0.4240],
          [ 0.5141,  0.5141,  0.5291,  ...,  0.4090,  0.4090,  0.3940],
          ...,
          [-1.2418, -1.0767,

In [ ]:
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

probs

tensor([[9.9289e-01, 5.9376e-03, 1.1761e-03],
        [9.9980e-01, 9.0017e-05, 1.0622e-04],
        [9.9907e-01, 4.5322e-04, 4.7207e-04]], grad_fn=<SoftmaxBackward0>)

In [ ]:
outputs

CLIPOutput(loss=None, logits_per_image=tensor([[24.1235, 19.0042, 17.3852],
        [26.1174, 16.8021, 16.9677],
        [24.0839, 16.3857, 16.4264]], grad_fn=<TBackward0>), logits_per_text=tensor([[24.1235, 26.1174, 24.0839],
        [19.0042, 16.8021, 16.3857],
        [17.3852, 16.9677, 16.4264]], grad_fn=<MulBackward0>), text_embeds=tensor([[ 0.0235,  0.0064, -0.0066,  ..., -0.0320, -0.0052, -0.0370],
        [-0.0047,  0.0017,  0.0025,  ..., -0.0632, -0.0084, -0.0294],
        [ 0.0299,  0.0122, -0.0279,  ..., -0.0260, -0.0330,  0.0043]],
       grad_fn=<DivBackward0>), image_embeds=tensor([[ 0.0352,  0.0099, -0.0230,  ...,  0.0089, -0.0337,  0.0154],
        [ 0.0471, -0.0131, -0.0189,  ...,  0.0518,  0.0369,  0.0157],
        [ 0.0190, -0.0280, -0.0265,  ...,  0.0905, -0.0348,  0.0026]],
       grad_fn=<DivBackward0>), text_model_output=BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 0.7443,  0.0374, 

In [ ]:
outputs.keys()


odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])

# Features de Texto

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

inputs = tokenizer(
    ["a photo of a cat", "a photo of a bird"],
    padding=True,
    return_tensors="pt"
    )

In [ ]:
inputs


{'input_ids': tensor([[49406,   320,  1125,   539,   320,  2368, 49407],
        [49406,   320,  1125,   539,   320,  3329, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
text_features = model.get_text_features(**inputs)
text_features.shape

torch.Size([2, 512])

# Features de Imagem

In [ ]:
inputs = processor(images=image, return_tensors="pt")
image_features = model.get_image_features(**inputs)
image_features.shape


torch.Size([1, 512])

In [ ]:
inputs


{'pixel_values': tensor([[[[-0.0696, -0.0259,  0.0179,  ...,  0.2223,  0.2077,  0.2223],
          [-0.0696, -0.0259,  0.0179,  ...,  0.4705,  0.4705,  0.4705],
          [-0.0842, -0.0405,  0.0033,  ...,  0.7333,  0.7333,  0.7479],
          ...,
          [ 0.1931,  0.1347,  0.0617,  ...,  0.6749,  0.6895,  0.6749],
          [ 0.4121,  0.3683,  0.3245,  ...,  0.7479,  0.7917,  0.8209],
          [ 0.8355,  0.7917,  0.7479,  ...,  0.7041,  0.7771,  0.7771]],

         [[ 0.3190,  0.3640,  0.4390,  ..., -0.3864, -0.3714, -0.3564],
          [ 0.3190,  0.3640,  0.4390,  ..., -0.1463, -0.1163, -0.1163],
          [ 0.3190,  0.3640,  0.4090,  ...,  0.1089,  0.1089,  0.1239],
          ...,
          [-0.3264, -0.3864, -0.4614,  ...,  0.4841,  0.4691,  0.4691],
          [-0.0562, -0.1012, -0.1463,  ...,  0.5591,  0.5891,  0.5741],
          [ 0.3790,  0.3340,  0.3040,  ...,  0.5291,  0.5291,  0.5441]],

         [[-0.4990, -0.4848, -0.4279,  ..., -0.4706, -0.4706, -0.4564],
          [-0